# Clonamos el repositorio con los modelos y herramientas

In [1]:
!git clone https://github.com/dannasalazar11/Msc_thesis.git

Cloning into 'Msc_thesis'...
remote: Enumerating objects: 259, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 259 (delta 55), reused 47 (delta 24), pack-reused 172 (from 1)
Receiving objects: 100% (259/259), 49.90 MiB | 43.23 MiB/s, done.
Resolving deltas: 100% (146/146), done.


In [2]:
import sys
sys.path.append('/kaggle/working/Msc_thesis')

from gmrrnet_adhd.utils import get_segmented_data, train_LOSO
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')

import tensorflow as tf
import numpy as np
import random
import os

# Establecer semilla
seed = 42

# Semillas para módulos principales
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

2025-04-30 14:45:03.309537: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746024303.555855      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746024303.616661      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Importar base de datos segmentada (Segmentos de 4 seg con translape del 50%, es decir, de 2 seg)

In [3]:
X, y, sbjs = get_segmented_data()
X.shape, y.shape, len(sbjs)

((8213, 19, 512), (8213, 2), 8213)

# Importamos el modelo y definimos los hiperparámetros

In [4]:
from tensorflow.keras.losses import CategoricalCrossentropy, MeanSquaredError
from tensorflow.keras.optimizers import Adam
from gmrrnet_adhd.models.EEGNet import EEGNet

model_name = 'EEGNet'
model_args = {
    'Chans' : 19,
    'Samples' : 512,
    'nb_classes': 2,
    'dropoutRate': 0.5,
    'kernLength': 32,
    'F1': 8,
    'D': 2,
    'F2': 16,
    'norm_rate': 0.25,
    'dropoutType': 'Dropout'
}

compile_args = {
    'loss': CategoricalCrossentropy(),
    'optimizer': lambda: Adam(1e-2),  # función que retorna un nuevo optimizador
    'metrics': ['categorical_accuracy']
}

model = EEGNet(**model_args)

model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1746024320.028399      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1746024320.029067      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ keras_tensor_2CLONE (InputLayer)     │ (None, 19, 512, 1)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Conv2D_1 (Conv2D)                    │ (None, 19, 512, 8)          │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 19, 512, 8)          │              32 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Depth_wise_Conv2D_1                  │ (None, 1, 512, 16)          │             304 │
│ (DepthwiseConv2D)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 1, 512, 16)          │              64 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 1, 512, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling2d (AveragePooling2D) │ (None, 1, 128, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1, 128, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Separable_Conv2D_1 (SeparableConv2D) │ (None, 1, 128, 16)          │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 1, 128, 16)          │              64 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_1 (Activation)            │ (None, 1, 128, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling2d_1                  │ (None, 1, 16, 16)           │               0 │
│ (AveragePooling2D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 1, 16, 16)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output (Dense)                       │ (None, 2)                   │             514 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ out_activation (Activation)          │ (None, 2)                   │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,746 (6.82 KB)

 Trainable params: 1,666 (6.51 KB)

 Non-trainable params: 80 (320.00 B)

# Resultados - Leave One Subject Out

In [5]:
results = train_LOSO(EEGNet, X, y, sbjs, model_args, compile_args, 1, 120, model_name)

Evaluando modelo para el sujeto #1: v107


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1746024326.209881      73 service.cc:148] XLA service 0x7a1c5400f310 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1746024326.210785      73 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1746024326.210804      73 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1746024326.704128      73 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1746024331.214623      73 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 243ms/step
Métricas para v107: {'accuracy': 1.0}

Evaluando modelo para el sujeto #2: v108


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 208ms/step
Métricas para v108: {'accuracy': 1.0}

Evaluando modelo para el sujeto #3: v109


...

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 375ms/step
Métricas para v60p: {'accuracy': 1.0}

Evaluando modelo para el sujeto #119: v6p


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 182ms/step
Métricas para v6p: {'accuracy': 0.9701492537313433}

Evaluando modelo para el sujeto #120: v8p


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 431ms/step
Métricas para v8p: {'accuracy': 1.0}

Resumen de métricas por sujeto:
Sujeto v107: {'accuracy': 1.0}
Sujeto v108: {'accuracy': 1.0}
Sujeto v109: {'accuracy': 0.9508196721311475}
Sujeto v10p: {'accuracy': 1.0}
Sujeto v110: {'accuracy': 1.0}
Sujeto v111: {'accuracy': 1.0}
Sujeto v112: {'accuracy': 1.0}
Sujeto v113: {'accuracy': 1.0}
Sujeto v114: {'accuracy': 1.0}
Sujeto v115: {'accuracy': 1.0}
Sujeto v116: {'accuracy': 1.0}
Sujeto v117: {'accuracy': 0.8787878787878788}
Sujeto v118: {'accuracy': 1.0}
Sujeto v120: {'accuracy': 1.0}
Sujeto v121: {'accuracy': 1.0}
Sujeto v123: {'accuracy': 1.0}
Sujeto v125: {'accuracy': 1.0}
Sujeto v127: {'accuracy': 1.0}
Sujeto v129: {'accuracy': 1.0}
Sujeto v12p: {'accuracy': 1.0}
Sujeto v131: {'accuracy': 1.0}
Sujeto v133: {'accuracy': 1.0}
Sujeto v134: {'accuracy': 1.0}
Sujeto v138: {'accuracy': 1.0}
Sujeto v140: {'accuracy': 1.0}
Sujeto v143: {'accuracy': 0.9830508474576272}
Sujeto v147: {'accuracy': 1.0}
Sujeto v1

In [6]:
results

{'v107': {'accuracy': 1.0},
 'v108': {'accuracy': 1.0},
 'v109': {'accuracy': 0.9508196721311475},
 'v10p': {'accuracy': 1.0},
 'v110': {'accuracy': 1.0},
 'v111': {'accuracy': 1.0},
 'v112': {'accuracy': 1.0},
 'v113': {'accuracy': 1.0},
 'v114': {'accuracy': 1.0},
 'v115': {'accuracy': 1.0},
 'v116': {'accuracy': 1.0},
 'v117': {'accuracy': 0.8787878787878788},
 'v118': {'accuracy': 1.0},
 'v120': {'accuracy': 1.0},
 'v121': {'accuracy': 1.0},
 'v123': {'accuracy': 1.0},
 'v125': {'accuracy': 1.0},
 'v127': {'accuracy': 1.0},
 'v129': {'accuracy': 1.0},
 'v12p': {'accuracy': 1.0},
 'v131': {'accuracy': 1.0},
 'v133': {'accuracy': 1.0},
 'v134': {'accuracy': 1.0},
 'v138': {'accuracy': 1.0},
 'v140': {'accuracy': 1.0},
 'v143': {'accuracy': 0.9830508474576272},
 'v147': {'accuracy': 1.0},
 'v149': {'accuracy': 1.0},
 'v14p': {'accuracy': 1.0},
 'v151': {'accuracy': 1.0},
 'v15p': {'accuracy': 1.0},
 'v173': {'accuracy': 1.0},
 'v177': {'accuracy': 1.0},
 'v179': {'accuracy': 1.0},
 'v

In [7]:
import pickle

with open(f'results_LOSO_{model_name}.pkl', 'wb') as f:
    pickle.dump(results, f)